In [1]:
from clumper import Clumper

def read_labels():

    path = "../data/truth/"
    annA = Clumper.read_csv(path + "annotatorA.csv", fieldnames=["id","ANN1"])
    annB = Clumper.read_csv(path + "annotatorB.csv", fieldnames=["id","ANN2"])
    annC = Clumper.read_csv(path + "annotatorC.csv", fieldnames=["id","ANN3"])
    majority = Clumper.read_csv(path + "majority.csv", fieldnames=["id","labelMajority"])

    result = annA.inner_join(annB, mapping={"id": "id"})
    result = result.inner_join(annC, mapping={"id": "id"})
    result = result.inner_join(majority, mapping={"id": "id"})

    return result

d = read_labels()

In [2]:
import pandas as pd
from pandas.api.types import CategoricalDtype

df = pd.DataFrame(d)

cats = ['none','weak','medium','strong']
t = pd.CategoricalDtype(categories=cats, ordered=True)

df['ANN1'] = df['ANN1'].astype(t) 
df['ANN2'] = df['ANN2'].astype(t)
df['ANN3'] = df['ANN3'].astype(t)

df.head(n=10)

,id,ANN1,ANN2,ANN3,labelMajority
0,607668877594497024,medium,none,medium,clickbait
1,607671137062010881,weak,weak,none,no-clickbait
2,607672151638876160,weak,none,weak,no-clickbait
3,607672568057700352,none,none,none,no-clickbait
4,607673643120918529,medium,weak,weak,no-clickbait
5,607674674168926209,medium,weak,weak,no-clickbait
6,607675444834398208,medium,none,none,no-clickbait
7,607675970632351744,none,none,none,no-clickbait
8,607678925410598913,medium,weak,medium,clickbait
9,607679294811144192,none,none,none,no-clickbait


In [3]:
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

df.head(n=10)

,id,ANN1,ANN2,ANN3,labelMajority
0,607668877594497024,2,0,2,clickbait
1,607671137062010881,1,1,0,no-clickbait
2,607672151638876160,1,0,1,no-clickbait
3,607672568057700352,0,0,0,no-clickbait
4,607673643120918529,2,1,1,no-clickbait
5,607674674168926209,2,1,1,no-clickbait
6,607675444834398208,2,0,0,no-clickbait
7,607675970632351744,0,0,0,no-clickbait
8,607678925410598913,2,1,2,clickbait
9,607679294811144192,0,0,0,no-clickbait


In [24]:
df.groupby(['ANN1','ANN2','ANN3']).agg('count').reset_index()

,ANN1,ANN2,ANN3,id,labelMajority
0,0,0,0,697,697
1,0,0,1,173,173
2,0,0,2,24,24
3,0,0,3,2,2
4,0,1,0,111,111
5,0,1,1,87,87
6,0,1,2,14,14
7,0,1,3,4,4
8,0,2,0,18,18
9,0,2,1,18,18


In [10]:
import altair as alt

df2 = df.groupby(['ANN1','ANN2','ANN3','labelMajority']).agg('count').reset_index()

df2.head()


,ANN1,ANN2,ANN3,labelMajority,id
0,0,0,0,no-clickbait,697
1,0,0,1,no-clickbait,173
2,0,0,2,no-clickbait,24
3,0,0,3,no-clickbait,2
4,0,1,0,no-clickbait,111


In [21]:
alt.Chart(df).mark_rect().encode(
    x='ANN1:O',
    y='ANN2:O',
    color='count()'
)

alt.Chart(...)

In [22]:
alt.Chart(df).mark_rect().encode(
    x='ANN1:O',
    y='ANN3:O',
    color='count()'
)

alt.Chart(...)

In [23]:
alt.Chart(df).mark_rect().encode(
    x='ANN2:O',
    y='ANN3:O',
    color='count()'
)

alt.Chart(...)

In [43]:
df['ANN1b'] = df['ANN1'].apply(lambda x: 'clickbait' if x > 1 else 'no-clickbait')
df['ANN2b'] = df['ANN2'].apply(lambda x: 'clickbait' if x > 1 else 'no-clickbait')
df['ANN3b'] = df['ANN3'].apply(lambda x: 'clickbait' if x > 1 else 'no-clickbait')

agg_binary = df.groupby(['ANN1b','ANN2b','ANN3b','labelMajority']).agg('count').reset_index()

agg_binary

,ANN1b,ANN2b,ANN3b,labelMajority,id,ANN1,ANN2,ANN3
0,clickbait,clickbait,clickbait,clickbait,305,305,305,305
1,clickbait,clickbait,no-clickbait,clickbait,141,141,141,141
2,clickbait,clickbait,no-clickbait,no-clickbait,1,1,1,1
3,clickbait,no-clickbait,clickbait,clickbait,288,288,288,288
4,clickbait,no-clickbait,no-clickbait,no-clickbait,546,546,546,546
5,no-clickbait,clickbait,clickbait,clickbait,33,33,33,33
6,no-clickbait,clickbait,no-clickbait,no-clickbait,82,82,82,82
7,no-clickbait,no-clickbait,clickbait,no-clickbait,107,107,107,107
8,no-clickbait,no-clickbait,no-clickbait,no-clickbait,1489,1489,1489,1489


In [44]:
lol = df[['ANN1','ANN2','ANN3']].transpose()
annot_list = lol.values.tolist()

In [45]:
import numpy as np
import krippendorff

print("Krippendorff's alpha for ordinal metric: ", krippendorff.alpha(reliability_data=annot_list,
                                                                      level_of_measurement="ordinal"))
#print("c's alpha for interval metric: ", krippendorff.alpha(reliability_data=annot_list))

Krippendorff's alpha for ordinal metric:  0.49461314289208724


In [46]:
from sklearn.metrics import cohen_kappa_score

# ordinal
print("ANN1 + ANN2: Cohen's kappa: ", cohen_kappa_score(df['ANN1'], df['ANN2']))
print("           ")
print("ANN2 + ANN3: Cohen's kappa: ", cohen_kappa_score(df['ANN2'], df['ANN3']))
print("           ")
print("ANN1 + ANN3: Cohen's kappa: ", cohen_kappa_score(df['ANN1'], df['ANN3']))
print("           ")

ANN1 + ANN2: Cohen's kappa:  0.19596580508358274
           
ANN2 + ANN3: Cohen's kappa:  0.29524094916608645
           
ANN1 + ANN3: Cohen's kappa:  0.22486472882848862
           


In [47]:
# binary 
print("ANN1 + ANN2: Cohen's kappa: ", cohen_kappa_score(df['ANN1b'], df['ANN2b']))
print("           ")
print("ANN2 + ANN3: Cohen's kappa: ", cohen_kappa_score(df['ANN2b'], df['ANN3b']))
print("           ")
print("ANN1 + ANN3: Cohen's kappa: ", cohen_kappa_score(df['ANN1b'], df['ANN3b']))
print("           ")

ANN1 + ANN2: Cohen's kappa:  0.30311220367503333
           
ANN2 + ANN3: Cohen's kappa:  0.39292003141524645
           
ANN1 + ANN3: Cohen's kappa:  0.40274574716811373
           


In [48]:
# binary vs. majority opinion
print("ANN1: Cohen's kappa: ", cohen_kappa_score(df['ANN1b'], df['labelMajority']))
print("           ")
print("ANN2: Cohen's kappa: ", cohen_kappa_score(df['ANN2b'], df['labelMajority']))
print("           ")
print("ANN3: Cohen's kappa: ", cohen_kappa_score(df['ANN3b'], df['labelMajority']))
print("           ")

ANN1: Cohen's kappa:  0.5831029063350888
           
ANN2: Cohen's kappa:  0.6435647633787802
           
ANN3: Cohen's kappa:  0.7793968208853785
           


Annotator 2 and 3 tend to agree, especially in binary (0.4 kappa). Annotator 1 has most "minority" view.